# 08 텍스트 분석
## 07 문서 군집화 소개와 실습(Opinion Review 데이터 세트)

### 문서 군집화 개념
문서 군집화(Document Clustering)는 비슷한 텍스트 구성의 문서를 군집화(Clustering)하는 것이다. 문서 군집화는 학습 데이터 세트가 필요 없는 비지도 학습 기반으로 동작한다.

### Opinion Review 데이터 세트를 이용한 문서군집화 수행하기

데이터 다운로드 : https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review 사이트에서 내려받을 수 있다.

해당 디렉터리 내에 있는 파일을 하나씩 읽어서 파일명과 파일 리뷰를 하나의 DataFrame으로 로드하여 파일명별로 어떤 리뷰를 담고 있는지 개략적으로 살펴본다. 이전 예제와 다르게 이번 예제는 데이터를 로드할 절대 경로 디렉터리를 먼저 지정한 뒤, 이 위치에서 데이터를 로딩한다.

해당 디렉터리 내의 모든 파일에 대해 각각 for 반복문으로 반복하면서 개별 파일명을 파일명 리스트에 추가하고 개별 파일은 DataFrame으로 읽은 후 다시 문자열로 반환한 뒤 파일 내용 리스트에 추가한다. 그리고 이렇게 만들어진 파일명 리스트와 파일 내용 리스트를 이용해 새롭게 파일명과 파일 내용을 칼럼으로 가지는 DataFrame을 생성한다.

In [1]:
import pandas as pd
import glob, os
import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 700)

# 아래는 제 컴퓨터에서 압축 파일을 풀어 놓은 디렉터리이니, 여러분의 디렉터리를 설정해 주세요  
path = r'C:\Users\chkwon\Text\OpinosisDataset1.0\OpinosisDataset1.0\topics'
# path로 지정한 디렉터리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합, 
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합 
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    
    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경. 
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]

    # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일 내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text


문서를 TF-IDF 형태로 피처 벡터화한다. TfidVectorizer는 Lemmatization 같은 어근 변환을 직접 지원하진 않지만, tokenizer 인자에 커스텀 어근 변환 함수를 적용해 어근 변환을 수행할 수 있다. 이를 위해 LemNormalize() 함수를 만든다.

In [2]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 입력으로 들어온 token단어들에 대해서 lemmatization 어근 변환. 
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# TfidfVectorizer 객체 생성 시 tokenizer인자로 해당 함수를 설정하여 lemmatization 적용
# 입력으로 문장을 받아서 stop words 제거-> 소문자 변환 -> 단어 토큰화 -> lemmatization 어근 변환. 
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

ValueError: empty vocabulary; perhaps the documents only contain stop words

문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬 데이터에 대해서 군집화를 수행해 어떤 문서끼리 군집되는지 확인해본다. 군집화 기법은 K-평균을 적용한다.

먼저 5개의 중심(Centroid) 기반으로 어떻게 군집화되는지 확인해 본다. 최대 반복횟수 max_iter는 10000으로 설정한다. KMeans를 수행한 후에 군집의 Label값과 중심별로 할당된 데이터 세트의 좌표 값을 구한다.

In [ ]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

각 데이터 별로 할당된 군집의 레이블을 파일명과 파일 내용을 가지고 있는 document_df DataFrame에 'cluster_label' 칼럼을 추가해 저장한다.

In [ ]:
document_df['cluster_label'] = cluster_label
document_df.head()

documnt_df DataFrame 객체에서 cluser_label로 어떤 파일명으로 매칭됐는지 보면서 군집화 결과를 확인해본다.

먼저 cluster_label=0인 데이터세트 확인

In [ ]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\구남이\AppData\Local\Temp\ipykernel_25840\546716908.py", line 1, in <module>
    document_df[document_df['cluster_label']==0].sort_values(by='filename')
NameError: name 'document_df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "c:\a

TypeError: object of type 'NoneType' has no len()

In [ ]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

In [ ]:

document_df[document_df['cluster_label']==2].sort_values(by='filename')


In [ ]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')


In [ ]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

중심 개수를 5개에서 3개로 낮춰서 그룹으로 군집화 한 뒤 결과 확인

In [ ]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

### 군집별 핵심 단어 추출하기

각 군집(Cluster)에 속한 문서는 핵심 단어를 주축으로 군집화돼 있을 것이다. 이번에는 각 군집을 구성하는 핵심 단어가 어떤 것이 있는지 확인해본다.

KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심(Centroid)을 기준으로 얼마나 가깝게 위치해 있는지 clusters_centers_ 라는 속성으로 제공한다. clusters_centres_는 배열 값으로 제공되며, 행은 개별 군집을, 열은 개별 피처를 의미한다.

군집 3개로 생성한 KMeans객체인 km_cluster에서 cluster_centers_ 속성값을 가져온 뒤 값을 확인해 본다.

In [ ]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centres_속성값을 이용해 각 군집별 핵심 단어를 찾아본다. cluster_centres_ 배열 내에서 가장 값이 큰 데이터의 위치 인덱스를 추출한 뒤, 해당 인덱스를 이용해 핵심 단어 이름과 그때의 상대 위치 값을 추출해 cluster_details라는 Dict 객체 변수에 기록하고 반환하는 것이 get_cluster_details()함수의 주요 로직이다.

In [ ]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

get_cluster_details()를 호출하면 dictionary를 원소로 가지는 리스트인 cluster_details를 반환한다. cluster_details에는 개별 군집번호, 핵심 단어, 핵심단어 중심 위치 상대값, 파일명 속성 값 정보가 있는데, 이를 좀 더 보기 좋게 표현하기 위해서 별도의 print_cluster_details() 함수를 만든다.

In [ ]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

위에서 생성한 get_cluster_details(), print_cluster_details()를 호출해 본다. get_cluster_detail() 호출 시 인자는 KMeans 군집화 객체, 파일명 추출을 위한 document_df DataFrame , 핵심 단어 추출을 위한 피처명 리스트, 전체 군집 개수, 그리고 핵심 단어 추출 개수이다. 피처명 리스트는 앞에서 TF-IDF 변환된 tfidf_vect 객체에서 get_feature_names()로 추출한다.

In [ ]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)